# Ejercicios 4

## Entrega Jueves 29 de Noviembre

1. Generar datos con “corrimiento al rojo” para los datos del Anillo en 2d. Para ello debemos crear mapa de velocidad: 
en una vecindad de $L/10$ de cada punto "pivote" en los datos, encontrar el punto más cercano y generar una velocidad 
para este punto pivote en la dirección del punto más cercano proporcional a su distancia $(v = 0.1dist)$. 
Guardar el nuevo punto con coordenadas

$$
\mathbf{r}=(x, y+v\cdot\hat{j})
$$

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import distros

In [3]:
n_points = 1000
r_int = 290.0
r_ext = 300.0
box_size = 1000.0
bin_size = 5.0

In [5]:
x_anillo, y_anillo = distros.anillo_dist(n_points, r_int, r_ext, box_size / 2.0, box_size / 2.0)

2. Cambiar el código de la función de correlación con estimador de Landy-Salay, para que sea anisotrópico y guarde la componente  de cada distancia en X y Y. 

   a) Con este código calcular la función de correlación anisotrópica del ejercicio anterior 
   y graficarla como gráfica de densidad en $2d$ (se puede usar imshow de matplotlib).
   
   b) (BONUS) Descomponer en la base de Legendre para encontrar el monopolo, cuadrupolo y hexadecapolo. 
   Graficarlos todos en un mismo plot. [Para realizar este ejercicio debes cambiar la función de correlación a coordenadas polares]